In [20]:
#! python3
# census2csv

# This script is used to scrape and consolidate demographic, population, and housing data 
# from the United States census website at: https://www.ffiec.gov/census/default.aspx

In [21]:
import csv
import os
import sys
import requests
from lxml import html
from bs4 import BeautifulSoup
from tkinter import filedialog
from nested_dict import nested_dict
import pandas as pd

In [22]:
# import specific county codes for parsing

geo_info = ([36047,36061,6075,11001,6037,17031,25025,36081,6081,25021,36005,6111,36085,36119,
            24033,53025,6059,34017,25017,6071,24031,6029,36059,42007,6065,6083,50003,6085,17197])

# standardize for use in parsing
state = []
county = []
for geo in geo_info:
    geo = str(geo)
    if len(geo) > 7:
        geo = geo[:-1].replace(".","") # remove the last digit, and remove decimals
    if len(geo) < 5:
        geo = '0' + geo
    state.append(geo[:2])
    county.append(geo[2:])

# define number of pages

# define columns for tables
columns = []
demographic_columns = ['Tract Code','Tract Income Level','Distressed (Y/N)','Tract Median Fam Income','MSA Median Fam Income','2017 Median Fam Income','2015 Median Fam Income','Tract Population','Tract Minority %','Minority Pop','Owner Occupied Units','1-4 Fam Units','state','county']
population_columns = ['Tract Code','Tract Population','Tract Minority %','Number of Families','Number Households','Non Hispanic White Pop','tract_minority_pop','american_indian_pop','asian_islander_pop','black_population','hisplanic_population','mixed_population','state','county']
housing_columns = ['Tract Code','Total Housing Units','1-4 Family Units','Median House Age','Inside City?','Owner Occupied Units','Vacant Units','Owner Occupied 1-4 Fam Units','Renter Occupied','state','county']
columns.append(demographic_columns)
columns.append(population_columns)
columns.append(housing_columns)

report = ['demographic','population','housing'] # type of data to pull from census site

In [48]:
def get_Page(url):
    # This function is used to pull the max page number from each counties report on the FFIEC website
    # Example of url for this function is: https://www.ffiec.gov/census/report.aspx?year=2017&county=019&tract=ALL&state=06&report=demographic
    html_response = requests.get(url).text.encode('utf-8') # request html (hot garbage)
    soup = BeautifulSoup(html_response, 'html.parser')
    pages = soup.find_all("span", class_="main-body") # page number sits in this class
    pages = str(pages[4].string) # specific to this site, lists page n of n
    try:
        page = int(pages[-2:]) # hoping this is a common format
    except Exception:
        page = 1
    return page

def consolidate_pages(state):
    pages = []
    for i in range(len(state)):
            # This uses the demographic report to pull the max page number, although any type of report should give the same result
            base_url = 'https://www.ffiec.gov/census/report.aspx?year=2017&state=' + state[i] + '&msa=&county=' + county[i] + '&tract=ALL&report=demographic'
            print(base_url)
            try:
                page = get_Page(base_url) # iterate through each state/county pair and pull the max page number
            except Exception: # if there is only one page, this exception will be thrown
                page = 1
            print(page)
            page = str(page)
            pages.append(page)
    return pages

In [52]:
report = ['demographic','population','housing']
baseurl = 'https://www.ffiec.gov/census/report.aspx?year=2017&county=019&tract=ALL&state=06&report=demographic'

for j in range(len(report)): 
    column_df = columns[j] # use the correct headers for each type of census report
    df = pd.DataFrame(columns = columns[j]) # initialize dataframe to hold consolidated reports
    for i in range(len(state)):
        print((i/((len(state)*len(report))))) # progress bar so I don't get impatient
        url = 'https://www.ffiec.gov/census/report.aspx?year=2017&state=' + state[i] + '&msa=&county=' + county[i] + '&tract=ALL&report=' + report[j]+ '&page=' + pages[i]
        html = requests.get(url).content
        df_list = pd.read_html(html)
        df_list = df_list[-1] # transpose to meet df structure
        df_list['state'] = state[i]
        df_list['county'] = county[i]
        df_list = df_list.iloc[1:,:] # remove first row to prevent duplicate column headers
        df_list.columns = column_df
        df = df.append(df_list) # add to data frame
        df.to_csv('censusInfo_' + report[j] + '.csv', index = False)

0.0
0.034482758620689655
0.06896551724137931
0.10344827586206896
0.13793103448275862
0.1724137931034483
0.20689655172413793
0.2413793103448276
0.27586206896551724
0.3103448275862069
0.3448275862068966
0.3793103448275862
0.41379310344827586
0.4482758620689655
0.4827586206896552
0.5172413793103449
0.5517241379310345
0.5862068965517241
0.6206896551724138
0.6551724137931034
0.6896551724137931
0.7241379310344828
0.7586206896551724
0.7931034482758621
0.8275862068965517
0.8620689655172413
0.896551724137931
0.9310344827586207
0.9655172413793104


In [ ]:

base_url = 'https://www.ffiec.gov/census/report.aspx?year=2017&state=' + state[i] + '&msa=&county=' + county[i] + '&tract=ALL&report=' + report[j]
#get_Page(url):
html_response = requests.get(base_url).text.encode('utf-8') # request html (hot garbage)
soup = BeautifulSoup(html_response, 'html.parser')
pages = soup.find_all("span", class_="main-body") # page number sits in this class
#table = soup.find_all('table')[0]
#elements = table.find_all("a")
#elements[0].a['href']
#elements.find_all("page")
pages = str(pages[4].string) # specific to this site, lists page n of n
#pages = (pages[4].contents)
print(page)
page = (pages[-2:]) # hoping this is a common format
page

In [ ]:
for st in state:
    dict[st] = {}
    for ct in county:
        url = 'https://www.ffiec.gov/census/report.aspx?year=2017&state=' + state[i] + '&msa=&county=' + county[i] + '&tract=ALL&report=demographic&page=1'
        html_response = requests.get(url).text.encode('utf-8') # request html (hot garbage)
        soup = BeautifulSoup(html_response, 'html.parser')
        pages = soup.find_all("span", class_="main-body") # page number sits in this class
        pages = str(pages[4].string) # specific to this site, lists page n of n
        page = int(pages[-2:]) # hoping this is a common format
        

In [ ]:
dict = {}
page = '1'
for st in state:
    dict[st] = {}
    for ct in county:
        dict[st][ct] = {page}

for key,value in dict.items():
    print(value)

In [10]:
#dict = {}
#for st in state:
#    dict[st] = {}
print(state, county)   
dict = {}
for i, j in zip(state, county):
    dict.setdefault(i, []).append(j)

dict

#dict['06']
#page = '1'
#for key,value in dict.items():
#    dict[value].append(nested)
#    for countyname in value:
#        nested = {countyname: page}
        
#dict'''

['36', '36', '06', '11', '06', '17', '25', '36', '06', '25', '36', '06', '36', '36', '24', '53', '06', '34', '25', '06', '24', '06', '36', '42', '06', '06', '50', '06', '17'] ['047', '061', '075', '001', '037', '031', '025', '081', '081', '021', '005', '111', '085', '119', '033', '025', '059', '017', '017', '071', '031', '029', '059', '007', '065', '083', '003', '085', '197']


{'06': ['075', '037', '081', '111', '059', '071', '029', '065', '083', '085'],
 '11': ['001'],
 '17': ['031', '197'],
 '24': ['033', '031'],
 '25': ['025', '021', '017'],
 '34': ['017'],
 '36': ['047', '061', '081', '005', '085', '119', '059'],
 '42': ['007'],
 '50': ['003'],
 '53': ['025']}